# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [23]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
from dotenv import load_dotenv
import os

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [ ]:
### 💡 Première requête guidée
BASE_URL = "http://api.openweathermap.org/data/2.5"
CITIES = ["Paris", "Berlin", "Madrid", "Rome", "Amsterdam", "Vienna"]
api_key = os.getenv('VC_key')
def get_weather_data(city, api_key):
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    # URL : current weather data
    url = f"{BASE_URL}/weather"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # Celsius
        'lang': 'fr'
    }

    # Votre code ici pour l'appel API
    # Gérez les erreurs HTTP !
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f'Erreur :  {response.status_code} : {response.text}')
    except requests.exceptions.RequestException as e:
        print(f'Erreur de connexion : {e}')



    return response.json()  




In [25]:
# Configuration API
API_KEY = os.getenv('OWM_key')
BASE_URL = "http://api.openweathermap.org/data/2.5"

# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Rome", "Amsterdam", "Vienna"]


def get_euro_weather(cities, API_KEY):
    url = f'{BASE_URL}/weather'
    results = []
    for city in cities:
        param = {
            'q': city,
            'appid': API_KEY,
            'units': 'metric',  # Celsius
            'lang': 'fr'
        }
        try:
            response = requests.get(url, params=param)
            if response.status_code == 200:
                data = response.json()
                results.append(data)
            else:
                print(f'Erreur :  {response.status_code} : {response.text}')
        except requests.exceptions.RequestException as e:
            print(f'Erreur de connexion : {e}')
    return results

list_weather = get_euro_weather(CITIES, API_KEY)

print(json.dumps(list_weather, indent=2))


Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Erreur :  401 : {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
[]


**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ? </br>
Vérifier si nous avons toutes les autorisations (si notre clé API nous permet de requêter)
- Comment gérer une ville introuvable ? </br>
Trouver une ville/localité proche
---

- Vérifier si nous avons toutes les autorisations (si notre clé API nous permet de requêter)
- Trouver la ville la plus proche répertoriée

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [26]:
# Configuration Visual Crossing
VC_API_KEY = os.getenv('VC_key')
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
start_date = '2025-06-30'
end_date = '2025-07-05'
CITIES = ["Paris", "Berlin", "Madrid", "Rome", "Amsterdam", "Vienna"]

def get_historical_weather(cities, start_date, end_date, api_key):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

    
    results = []
    for city in cities:
        url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"
        params = {
            'key': api_key,
            'include': 'days',
            'unitGroup': 'metric',
            'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                results.append({'ville': city, 'data': data})
            else:
                print(f'Erreur :  {response.status_code} : {response.text}')
        except requests.exceptions.RequestException as e:
            print(f'Erreur de connexion : {e}')
    return results


rentree_weather = get_historical_weather(CITIES, start_date, end_date, VC_API_KEY)

# print(json.dumps(rentree_weather, indent=2, ensure_ascii=False))

villes = [] # liste de dictionnaires de données de chaque ville (chaque dict = une ligne)
for villes_data in rentree_weather: 
    ville = villes_data['ville'] # récupération des noms de villes
    for day in villes_data['data']['days']: # récupération de chaque jour
 # Mise en relation des clés/valeurs pour chaque données récoltées
        villes.append({ 
            'date': day['datetime'],
            'ville': ville,
            'temp': day['temp'],
            'humidity': day['humidity'],
            'precip': day['precip'],
            'windspeed': day['windspeed']
        })
# Création du Dataframe
df_rentree_weather = pd.DataFrame(villes)
# Vérification
print(df_rentree_weather.head(10))




Erreur :  401 : No session info found.
Erreur :  401 : No session info found.
Erreur :  401 : No session info found.
Erreur :  401 : No session info found.
Erreur :  401 : No session info found.
Erreur :  401 : No session info found.
Empty DataFrame
Columns: []
Index: []


### 🎯 Mission pratique
Récupérez les données des 30 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [ ]:
### 🗺️ Obtenir les coordonnées
def get_city_coordinates(cities, api_key):
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """
    url = 'http://api.openweathermap.org/geo/1.0/direct'
    city_coordinates = []
    for city in cities:
        params = {
            'q': city,
            'limit': 1,
            'appid': api_key,
            'lang': 'fr'
        }
        try:
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                if data:
                    lat = data[0]['lat']
                    lon = data[0]['lon']
                    city_coordinates.append({
                        'ville': city,
                        'lat': lat,
                        'lon': lon
                    })
                else:
                    print('erreur')
        except requests.exceptions.RequestException as e:
            print(f'erreur : {e}')
    return city_coordinates


coords = get_city_coordinates(CITIES, API_KEY)
print(json.dumps(coords, indent=2, ensure_ascii=False))

In [16]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
def get_air_quality(lat, lon, api_key):
    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """
    url = 'http://api.openweathermap.org/data/2.5/air_pollution'
    params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key    
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data and 'list' in data and len(data['list']) > 0:
                aqi = data['list'][0]['main']['aqi']
                composants = data['list'][0]['components']
                return {
                    'lat': lat,
                    'lon': lon,
                    'aqi': aqi,
                    'composants': composants
                }
            else:
                print('Données manquantes.')
    except requests.exceptions.RequestException as e:
        print(f'Erreur : {e} ')

    return None

In [22]:
# Test get_air_quality
city_coords = get_city_coordinates(CITIES, API_KEY)
air_quality = []

for city in city_coords:
    ville = city['ville']
    lat = city['lat']
    lon = city['lon']
    aqi_data = get_air_quality(lat, lon, API_KEY)
    if aqi_data:
        aqi_data['ville'] = ville
        air_quality.append(aqi_data)

print(json.dumps(air_quality, indent=2, ensure_ascii=False))

[
  {
    "lat": 48.8588897,
    "lon": 2.3200410217200766,
    "aqi": 1,
    "composants": {
      "co": 151.94,
      "no": 0.81,
      "no2": 2.36,
      "o3": 55.93,
      "so2": 0.78,
      "pm2_5": 3.11,
      "pm10": 4.5,
      "nh3": 2.17
    },
    "ville": "Paris"
  },
  {
    "lat": 52.5170365,
    "lon": 13.3888599,
    "aqi": 1,
    "composants": {
      "co": 117.16,
      "no": 0.51,
      "no2": 3.6,
      "o3": 33.27,
      "so2": 0.7,
      "pm2_5": 0.98,
      "pm10": 1.22,
      "nh3": 1.23
    },
    "ville": "Berlin"
  },
  {
    "lat": 40.4167047,
    "lon": -3.7035825,
    "aqi": 1,
    "composants": {
      "co": 92.77,
      "no": 0.92,
      "no2": 3.14,
      "o3": 55.19,
      "so2": 0.08,
      "pm2_5": 2.41,
      "pm10": 4.2,
      "nh3": 8.7
    },
    "ville": "Madrid"
  },
  {
    "lat": 41.8933203,
    "lon": 12.4829321,
    "aqi": 2,
    "composants": {
      "co": 110.77,
      "no": 0.06,
      "no2": 0.35,
      "o3": 77.81,
      "so2": 0.5,
   


## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [ ]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [ ]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [ ]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites